In [1]:
import tensorflow as tf
from osgeo import gdal, gdalconst
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [2]:
driver = gdal.GetDriverByName("HFA")
driver.Register()

6

In [3]:
# load files
needclass_file = r"G:\Paper\JL\GF-2_3000_GS_302.dat"
ds = gdal.Open(needclass_file)
# getting dimensions
cols = ds.RasterXSize
rows = ds.RasterYSize
bands = ds.RasterCount
print(cols, rows, bands)
geotransform = ds.GetGeoTransform()
originX = geotransform[0]
originY = geotransform[3]
print(geotransform)

301 301 4
(543616.671, 0.96, -0.0, 2952995.761, -0.0, -0.96)


In [4]:
# read feature files by block
needclassdsbypixel = []
m = 9
for i in range(0, rows, 1):
    if i + m <= rows:
        for j in range(0, cols, 1):
            if j + m <= cols:
                data = ds.ReadAsArray(j, i, m, m).reshape(m*m*4,)/10000
                needclassdsbypixel.append(data)

In [5]:
print( needclassdsbypixel[0],  needclassdsbypixel[0].shape)

[ 0.0465  0.047   0.0477  0.0488  0.0498  0.0498  0.0483  0.0473  0.0461
  0.0471  0.0495  0.0509  0.051   0.0502  0.0486  0.0472  0.0459  0.0441
  0.0458  0.0474  0.0489  0.0494  0.0485  0.0454  0.0435  0.0439  0.0455
  0.047   0.0466  0.0472  0.0475  0.0459  0.042   0.0406  0.0424  0.0459
  0.0494  0.0485  0.0493  0.049   0.0461  0.0429  0.0426  0.044   0.0451
  0.0508  0.0499  0.0501  0.0491  0.0465  0.045   0.0459  0.0462  0.0444
  0.0523  0.0505  0.0483  0.0462  0.0464  0.0481  0.0487  0.0466  0.0437
  0.0528  0.0499  0.0463  0.0441  0.0465  0.0503  0.0498  0.0462  0.0436
  0.0521  0.0501  0.0484  0.0464  0.0464  0.0476  0.047   0.0455  0.0447
  0.0637  0.0646  0.0656  0.0674  0.0691  0.069   0.0667  0.0655  0.0642
  0.0657  0.0701  0.0727  0.0726  0.0708  0.0678  0.0652  0.0633  0.0608
  0.0643  0.0674  0.0703  0.0708  0.0686  0.0625  0.059   0.06    0.0632
  0.0668  0.0663  0.0674  0.0676  0.064   0.0565  0.0536  0.057   0.0636
  0.0715  0.0696  0.0706  0.0695  0.0637  0.0574  0

In [6]:
# define function
def variable_summaries(var):
    with tf.name_scope("summaries"):
        mean = tf.reduce_mean(var)
        tf.summary.scalar("mean", mean)
        with tf.name_scope("stddev"):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar("stddev", stddev)
        tf.summary.scalar("max", tf.reduce_max(var))
        tf.summary.scalar("min", tf.reduce_min(var))
        tf.summary.histogram("histogram", var)

# init weights function
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
#     initial = tf.contrib.layers.variance_scaling_initializer()
    return tf.Variable(initial)

# init biases function
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# define convo layer function
def conv2d(x,W):
    # x input tensor of shape [batch, in_height, in_width, in_channels]
    # W filter/kernel tensor of shape [filter_height, filter_width, in_channels,out_channels]
    #strides[0]=strides[3]=1,strides[1]代表x方向的步长，strides[2]代表y方向的步长
    #padding: A "string" from:"SAME", "VALID"
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")
def conv3d(x,W):
    # x input tensor of shape [batch, in_depth,in_height, in_width, in_channels]
    # W filter/kernel tensor of shape [filter_depth,filter_height, filter_width, in_channels,out_channels]
    #strides[0]=strides[4]=1,strides[1]代表x方向的步长，strides[2]代表y方向的步长,strides[3]代表z方向的步长
    #padding: A "string" from:"SAME", "VALID"
    return tf.nn.conv3d(x, W, strides=[1, 1, 1, 1,1], padding="SAME")
# define pooling layer
def max_pool_2x2(x):
    #ksize(1, x, y, 1)
    return tf.nn.max_pool(x,ksize=[1, 2, 2, 1], strides= [1, 2, 2, 1], padding="SAME")


with tf.name_scope("input"):
# make placeholder
    x = tf.placeholder(tf.float32, [None,m*m*4], name="x")
    y = tf.placeholder(tf.float32, [None,5], name="y")
    x_image = tf.reshape(x, [-1, m, m, 4])

with tf.name_scope("conv1"):
    # initial first conv layer weights and biases
    with tf.name_scope("W_conv1"):
        W_conv1 = weight_variable([3, 3, 4, 16])
        variable_summaries(W_conv1)
    with tf.name_scope("b_conv1"):
        b_conv1 = bias_variable([16])
        variable_summaries(b_conv1)
    # x_image and weights biases  conv1, relu activation function
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
with tf.name_scope("conv2"):
    # initial second conv layer weights and biases
    with tf.name_scope("W_conv2"):
        W_conv2 = weight_variable([3, 3, 16, 32])
        variable_summaries(W_conv2)
    with tf.name_scope("b_conv2"):
        b_conv2 = bias_variable([32])
        variable_summaries(b_conv2)
    # h_pool1 and weights biases  conv2, relu activation function
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2) 
with tf.name_scope("fc1"):
    # initial first full-connectelly layer weights and biases
    with tf.name_scope("W_fc1"):
        W_fc1 = weight_variable([3*3*32, 1000])
        variable_summaries(W_fc1)
    with tf.name_scope("b_fc1"):
        b_fc1 = bias_variable([1000])
        variable_summaries(b_fc1)
    #convert pool layer into 1D
    h_pool2_flat = tf.reshape(h_pool2,[-1, 3*3*32])

    # get first full-connect layer output
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
with tf.name_scope("dropout"):
    # keep_prob and dropout
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

with tf.name_scope("softmax"):
    # initial second full-connect layer weights and biases
    with tf.name_scope("W_softmax"):
        W_fc2 = weight_variable([1000, 5])
        variable_summaries(W_fc2)
    with tf.name_scope("b_softmax"):
        b_fc2 = bias_variable([5])
        variable_summaries(b_fc2)
    # finally output
    prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)
# cross-entropy
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
    tf.summary.scalar("loss",loss)
with tf.name_scope("train"):   
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

with tf.name_scope("accuracy"): 
    # accuracy
    with tf.name_scope("correct_prediction"):  
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
    with tf.name_scope("accuracy"):  
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar("accuracy",accuracy)

# initial variables
init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess,r"G:\Paper\python\TensorFlow\test06\cnn_9x9lr=1000=1e-4-2c_kp==0.5/my_net.ckpt")
    da_pre = sess.run(prediction, feed_dict= {x:needclassdsbypixel,keep_prob:1.0})
    print(da_pre)

INFO:tensorflow:Restoring parameters from G:\Paper\python\TensorFlow\test06\cnn_9x9lr=1000=1e-4-2c_kp==0.5/my_net.ckpt
[[  8.39084672e-19   9.56588367e-04   5.11351449e-04   2.22618833e-11
    9.98532057e-01]
 [  9.28398956e-20   4.17122897e-03   1.17073278e-03   9.34125867e-12
    9.94658053e-01]
 [  3.11420511e-20   1.36385416e-03   9.54934570e-04   1.56043798e-12
    9.97681141e-01]
 ..., 
 [  2.10194356e-13   1.52006690e-13   9.99928117e-01   3.72752813e-18
    7.18688971e-05]
 [  8.55841954e-14   6.04491717e-13   9.99545753e-01   1.46791769e-17
    4.54222376e-04]
 [  1.30177729e-13   2.68159979e-13   9.99695301e-01   1.43803943e-17
    3.04753601e-04]]


In [7]:
class_result1 = np.argmax(da_pre, axis=-1)
class_result2 = class_result1.reshape(301-m+1, 301-m+1) 
class_result2

array([[4, 4, 4, ..., 4, 4, 4],
       [4, 4, 4, ..., 4, 4, 4],
       [4, 4, 4, ..., 4, 4, 4],
       ..., 
       [1, 1, 1, ..., 2, 2, 2],
       [1, 1, 1, ..., 2, 2, 2],
       [2, 1, 1, ..., 2, 2, 2]], dtype=int64)

In [20]:
outDataset = driver.Create(r"G:\Paper\JL\test_res\302_cnn_9x9_1000_cl.img",
                          301-m+1, 301-m+1, 1, gdal.GDT_Int32)

In [21]:
outBand = outDataset.GetRasterBand(1)
outBand.WriteArray(class_result2,0, 0)
A = outBand.ReadAsArray(0,0, 20, 20)
A

array([[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 1, 1],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 1, 1],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 1, 1],
       [4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 1],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 1, 1],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 2, 2, 2, 1, 1, 1],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1],
       [4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1],
       [4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 4, 4, 1, 1, 1, 1, 1, 1, 1],
       [4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4],
       [1, 1, 1, 1, 1, 1, 1, 1, 1,

In [22]:
outBand.FlushCache()
outBand.GetStatistics(0, 1)

[0.0, 4.0, 2.4091369730573096, 1.3475545220881857]

In [23]:
geoTransform = ds.GetGeoTransform()
new_geoTransform=(geoTransform[0]+(0.96*(m-1)/2),geoTransform[1],geoTransform[2],geoTransform[3]-(0.96*(m-1)/2),geoTransform[4],geoTransform[5])
outDataset.SetGeoTransform(new_geoTransform)
proj = ds.GetProjection()
outDataset.SetProjection(proj)

0

In [24]:
# building pyramids
gdal.SetConfigOption("HFA_USE_RRD", "YES")
outDataset.BuildOverviews(overviewlist=[2, 4, 8, 16, 32, 64, 128])

0

In [25]:
geotransform = outDataset.GetGeoTransform()
print(geotransform[0])
proj = outDataset.GetProjection()
print(proj)

543620.5109999999
PROJCS["WGS_1984_UTM_Zone_50N",GEOGCS["GCS_WGS_1984",DATUM["WGS_1984",SPHEROID["WGS_84",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",117.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1]]
